---
# Final Project Artificial Intelligence <br>Max Paulenz (A12921301), Friedrich Wenisch (A12921323), Jannis Kaliske (A12921305)
Traveling Salesman Problem applied to PCB hole drilling<br>
TabuSearch - MetaHeuristic
---

---

In [ ]:
import time
import threading
from InputData import *
from EvaluationLogic import *
from OutputData import *
from Solver import *
from BestStartSolution import *

# Function to process each scenario from start index to end index
def process_scenario(start, end):
    for i in range(start, end):
        if i == 0:
            continue  # Skip processing for index 0 if needed

        start_time = time.time()  # Start timing the process

        # Construct the file path for the scenario's input data
        path = 'scenario_example_id_' + str(i) + '.json'
        seedNumber = 50  # Seed number for generating initial solution
        maxTabuListLength = 35  # Maximum length of the tabu list
        maxIterations = 500  # Maximum number of iterations for the tabu search

        # Load input data from the specified path
        data = InputData(path)
        # Generate the best start solution using the provided seed number
        firstSolution = BestStartSolution(data.matrix, seedNumber).generate_best_start_solution()

        # Perform the tabu search to find the optimal solution
        Solver(path).tabuSearch(firstSolution, data.matrix, data.coordinates, path, maxTabuListLength, maxIterations)

        end_time = time.time()  # End timing after the process
        duration = end_time - start_time  # Calculate the duration of the process

        # File path for storing solutions
        file_path = 'Solutions\Solution-' + path.replace('.json', '') + '.csv'

        # Read existing data from the CSV file
        with open(file_path, mode='r', newline='') as file:
            reader = csv.reader(file, delimiter=';')
            data = list(reader)

        # Append the new duration to the data
        data.append([str(duration)])

        # Write the updated data back to the CSV file
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file, delimiter=';')
            writer.writerows(data)

        # Print the duration for each iteration
        print(f"Iteration {i}: Duration {duration:.2f} seconds")

# Create threads for parallel processing of scenarios
threads = []
for i in range(0, 16):
    t = threading.Thread(target=process_scenario, args=(i*16, i*16+1,))
    t.start()
    threads.append(t)

# Wait for all threads to complete
for t in threads:
    t.join()

In [ ]:
import os
import pandas as pd

def datatraining():
    # Define the column names manually to be used in the DataFrame
    column_names = ['Id', 'Coordinates', 'Order', 'Distance', 'Time']

    # Directory containing the CSV files
    directory = 'Solutions'

    # Loop through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):  # Check if the file is a CSV
            # Construct the full file path
            file_path = os.path.join(directory, filename)

            # Load the CSV file without using any row as header and specifying column names
            data = pd.read_csv(file_path, delimiter=';', index_col=None, header=None)
            data = data.T  # Transpose the data to switch rows and columns
            data.columns = column_names  # Set the column names to the DataFrame

            # Construct the output file path by modifying the filename
            output_file_path = 'Datatraining/Datatraining_' + filename.split('_')[-1]

            # Export the DataFrame to a new CSV file
            data.to_csv(output_file_path, index=False)

            # Print the filename and the path where it was saved
            print(f'Processed {filename} and saved to {output_file_path}')

# Call the function to process the data
datatraining()